```
추천시스템 만들기 - 엔카 차량(중고차) 추천

22.02.09

1. 데이터 수집(크롤링) : 엔카 중고차 평점 및 닉네임 수집
 - 유저 아이디 
 - 차량 아이디 (차량이름)
 - 평점 (별점 - 숫자로 환산필요)

2. 코사인 유사도 + CF-KNN (아이템 기반 인접 이웃 협업 필터링)


3. 유사도 높은 순으로 출력

-----------------------------------------------------------------------
```

마지막에 surprise로 rmse 출력

# 데이터 수집(크롤링) : SK엔카 중고차 평점 및 추천

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import chromedriver_autoinstaller
import warnings
warnings.filterwarnings('ignore')

In [278]:
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

driver.get('http://www.encar.com/mocha.do?WT.hit=index_contents_title')
time.sleep(1) 

In [279]:
# 더보기
try : 
    for more in range(10):
        more_page = driver.find_element_by_css_selector('.btn_info_more')
        more_page.click()   
except:
    pass

In [280]:
#url 수집
product_url_list = []

product = ".link_car"
product_raw = driver.find_elements_by_css_selector(product)
product_raw[0]

<selenium.webdriver.remote.webelement.WebElement (session="9f93fc1a3d2f8734d1550bbe53f76f1d", element="81ecf0a7-4aae-4c71-a170-bac72efa71c5")>

In [281]:
#title 수집
product_title_list = []

product2 = ".link_car .tit_car"
product_raw2 = driver.find_elements_by_css_selector(product2)
product_raw2[0]

<selenium.webdriver.remote.webelement.WebElement (session="9f93fc1a3d2f8734d1550bbe53f76f1d", element="e885e50b-1128-42cc-8624-97dd4c3be69f")>

In [282]:
product_raw2[0].text

'현대 아반떼 (CN7)'

In [283]:
# 크롤링한 url 정제 시작
for i in product_raw:
    url = i.get_attribute('href')
    product_url_list.append(url)
time.sleep(1)
    
# 제목 크롤링 시작    
for j in product_raw2:
    title = j.text
    product_title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(product_url_list))
print('title갯수: ', len(product_title_list))

현대 아반떼 (CN7)
볼보 S60 3세대
BMW M2 (F87)
현대 싼타페 TM
BMW 5시리즈 (G30)
벤츠 S-클래스 W222
포르쉐 카이맨
포르쉐 911
현대 LF 쏘나타 하이브리드
기아 K8
현대 더 뉴 싼타페
아우디 A5 (F5)
현대 아이오닉5
르노삼성 더 뉴 SM6
현대 코나 하이브리드
르노삼성 XM3
쉐보레(GM대우) 트레일블레이저
기아 셀토스
기아 K5 3세대
기아 모하비 더 마스터
기아 쏘울 부스터
기아 더 뉴 니로
기아 스토닉
현대 쏘나타 (DN8)
기아 더 K9
현대 아슬란
기아 올 뉴 K3
현대 아반떼 AD
제네시스 G70
기아 스팅어
현대 그랜저 IG 하이브리드
볼보 V90 크로스컨트리
볼보 S90
벤츠 E-클래스 W213
기아 K5 하이브리드 2세대
포르쉐 718 박스터
BMW 3시리즈 (F30)
BMW 5시리즈 (F10)
현대 팰리세이드
폭스바겐 티구안 2세대
현대 스타리아
폭스바겐 티록
제네시스 G80 (RG3)
쉐보레(GM대우) 볼트 EV
쉐보레(GM대우) 트래버스
미니 쿠퍼 S 컨버터블
쉐보레(GM대우) 올 뉴 크루즈
쉐보레(GM대우) 콜로라도
현대 베뉴
쉐보레(GM대우) 더 뉴 말리부
지프 랭글러 (JL)
기아 더 뉴 K5 2세대
현대 올 뉴 투싼
도요타 RAV4
현대 더 뉴 아반떼 AD
기아 스포티지 더 볼드
현대 벨로스터 (JS)
기아 더 뉴 쏘렌토
기아 스포티지 4세대
현대 코나
현대 i30 (PD)
기아 니로
쉐보레(GM대우) 올 뉴 카마로
현대 쏘나타 뉴 라이즈
기아 올 뉴 카니발
BMW 1시리즈 (F20)
기아 K5 2세대
기아 올 뉴 K7
현대 그랜저 IG
벤츠 GLC-클래스 X253
벤츠 C-클래스 W205
포르쉐 마칸
렉서스 뉴 ES300h
쉐보레(GM대우) 올 뉴 말리부
르노삼성 QM6
제네시스 G80
현대 더 뉴 맥스크루즈
현대 싼타페 더 프라임
인피니티 Q50
렉서스 CT200h
현대 더 뉴 i30
제네시스 EQ900
현대 NF 쏘나타 트랜스폼
현대 올 뉴 투싼
아우디 뉴 A6
현대 LF 쏘나타
현

In [284]:
df = pd.DataFrame({'url':product_url_list, 'title':product_title_list})
df

,url,title
0,http://www.encar.com/mocha/mochacontents.do?me...,현대 아반떼 (CN7)
1,http://www.encar.com/mocha/mochacontents.do?me...,볼보 S60 3세대
2,http://www.encar.com/mocha/mochacontents.do?me...,BMW M2 (F87)
3,http://www.encar.com/mocha/mochacontents.do?me...,현대 싼타페 TM
4,http://www.encar.com/mocha/mochacontents.do?me...,BMW 5시리즈 (G30)
...,...,...
176,http://www.encar.com/mocha/mochacontents.do?me...,현대 뉴 아반떼 XD
177,http://www.encar.com/mocha/mochacontents.do?me...,르노삼성 뉴SM3
178,http://www.encar.com/mocha/mochacontents.do?me...,쌍용 코란도 스포츠
179,http://www.encar.com/mocha/mochacontents.do?me...,현대 뉴 그랜저 XG


In [285]:
# csv파일로 저장
df.to_csv("sk_encar_url.csv", encoding = 'utf-8-sig')

In [286]:
# "url_list.csv" 불러오기
url_load = pd.read_csv("sk_encar_url.csv")
url_load = url_load.drop("Unnamed: 0", axis=1)
url_load

,url,title
0,http://www.encar.com/mocha/mochacontents.do?me...,현대 아반떼 (CN7)
1,http://www.encar.com/mocha/mochacontents.do?me...,볼보 S60 3세대
2,http://www.encar.com/mocha/mochacontents.do?me...,BMW M2 (F87)
3,http://www.encar.com/mocha/mochacontents.do?me...,현대 싼타페 TM
4,http://www.encar.com/mocha/mochacontents.do?me...,BMW 5시리즈 (G30)
...,...,...
176,http://www.encar.com/mocha/mochacontents.do?me...,현대 뉴 아반떼 XD
177,http://www.encar.com/mocha/mochacontents.do?me...,르노삼성 뉴SM3
178,http://www.encar.com/mocha/mochacontents.do?me...,쌍용 코란도 스포츠
179,http://www.encar.com/mocha/mochacontents.do?me...,현대 뉴 그랜저 XG


# 크롤링 준비

In [250]:
j = 1

url = url_load['url'][j]
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.get(url) 

In [55]:
# 댓글 클릭
driver.find_element_by_link_text('댓글').click()
time.sleep(1)

In [211]:
# 총 댓글 수 크롤링 
total_comment_sel = driver.find_element_by_css_selector('.tit_cmt')
total_comment = int(total_comment_sel.text.split(' ')[1].replace('개',''))
total_comment

32

In [212]:
# 별점 크롤링 
result_1 = driver.find_elements_by_css_selector('.list_comment .img_mocha')
result_1

stars = result_1[ 2*j +1 ].get_attribute('style')
stars
score = int(stars.split(' ')[1].replace('%;' , '')) * 0.05
score

4.5

In [241]:
# 닉네임 크롤링 
nick = driver.find_elements_by_css_selector('.list_comment .txt_user')
len(nick)

32

In [242]:
nick[19].text

'choidaniel87'

In [243]:
nick_id = nick[19].text.split(' ')[0]
nick_id

'choidaniel87'

In [200]:
# 더보기 누르는 건데 없어도 될듯>?
try : 
    more_page_num = total_comment // 10  - 1
    
    for more in range(a):
        more_page = driver.find_element_by_css_selector('.btn_info_more')
        more_page.click()
        
except:
    pass

닉네임(nick_id) / 차량 이름(title) / 평점(score) 크롤링 완료

# 크롤링 시작 (for문)

In [288]:
%%time

title_list = []
score_list = []
nick_id_list = []

number = 181

for i in tqdm_notebook(range(0, number)):
    url = url_load['url'][i]
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    driver.get(url) 

    # 댓글 클릭
    driver.find_element_by_link_text('댓글').click()
    time.sleep(1)


    try : 
        for more in range(10):
            more_page = driver.find_element_by_css_selector('.btn_info_more')
            more_page.click()   
    except:
        pass

    time.sleep(1)

    nick = driver.find_elements_by_css_selector('.list_comment .txt_user')
    
    # 제목 크롤링 시작    
    
        
    for k in range(len(nick)):

        #title
        title = url_load['title'][i]

        # 별점 크롤링 
        try:
            result_1 = driver.find_elements_by_css_selector('.list_comment .img_mocha')
            stars = result_1[ 2*k +1 ].get_attribute('style')
            score = int(stars.split(' ')[1].replace('%;' , '')) * 0.05
        except :
            pass

        # 닉네임 크롤링 
        try:
            nick_id = nick[k].text.split(' ')[0]

        except:
            pass
        
                
        #딕셔너리 저장 
        title_list.append(title)
        nick_id_list.append(nick_id)
        score_list.append(score)       

    driver.close()

  0%|          | 0/181 [00:00<?, ?it/s]

Wall time: 25min 6s


In [289]:
sk_encar_df = pd.DataFrame()

sk_encar_df['title'] = title_list
sk_encar_df['id'] = nick_id_list
sk_encar_df['rating'] = score_list

print(sk_encar_df.shape)
sk_encar_df.head(5)

(1903, 3)


,title,id,rating
0,현대 아반떼 (CN7),alexius,4.5
1,현대 아반떼 (CN7),bluebird7788,4.5
2,현대 아반떼 (CN7),asdfasdf724,4.0
3,현대 아반떼 (CN7),dudwo6085,2.0
4,현대 아반떼 (CN7),ycd3456,5.0


In [291]:
sk_encar_df.to_csv("encar.csv", encoding='utf-8-sig')

# 추천 시스템 작성

In [5]:
data = pd.read_csv('encar.csv')
data = data.drop('Unnamed: 0', axis = 1)

In [6]:
data.head()

,title,id,rating
0,현대 아반떼 (CN7),alexius,4.5
1,현대 아반떼 (CN7),bluebird7788,4.5
2,현대 아반떼 (CN7),asdfasdf724,4.0
3,현대 아반떼 (CN7),dudwo6085,2.0
4,현대 아반떼 (CN7),ycd3456,5.0


In [22]:
data.title.nunique()  # 홈페이지에서 제공하는 차량의 수는 181대이지만 리뷰가 없는 차량이 5대 존재하여 176대만 크롤링.

176

### 사용자-아이템 평점 행렬로 변환

In [11]:
# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = data.pivot_table('rating', index='id', columns='title')
ratings_matrix

title,BMW 1시리즈 (F20),BMW 3시리즈 (F30),BMW 5시리즈 (F10),BMW 5시리즈 (G30),BMW M2 (F87),BMW Z4 (E89),기아 K3,기아 K5,기아 K5 2세대,기아 K5 3세대,...,현대 올 뉴 투싼,현대 제네시스,현대 제네시스 DH,현대 제네시스 쿠페,현대 코나,현대 코나 하이브리드,현대 투스카니,현대 투싼 ix,현대 팰리세이드,혼다 New CR-V
id,,,,,,,,,,,,,,,,,,,,,
Baestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bestsoul4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bosshuga,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Button0308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chdnet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zksdk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zksvk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zpsemf925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

title,BMW 1시리즈 (F20),BMW 3시리즈 (F30),BMW 5시리즈 (F10),BMW 5시리즈 (G30),BMW M2 (F87),BMW Z4 (E89),기아 K3,기아 K5,기아 K5 2세대,기아 K5 3세대,...,현대 올 뉴 투싼,현대 제네시스,현대 제네시스 DH,현대 제네시스 쿠페,현대 코나,현대 코나 하이브리드,현대 투스카니,현대 투싼 ix,현대 팰리세이드,혼다 New CR-V
id,,,,,,,,,,,,,,,,,,,,,
Baestyle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bestsoul4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bosshuga,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Button0308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chdnet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zksdk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zksvk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zpsemf925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 아이템-사용자 평점 행렬로 변환

In [13]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()  # 전치 행렬

print(ratings_matrix_T.shape)
ratings_matrix_T.head(5)

(176, 1034)


id,Baestyle,Bestsoul4,Bosshuga,Button0308,Chdnet,Choivs0,DreamLover,GONZA_admin,Giddens,HMCTJ,...,zephyr97,zerocool4u,zizilub,zizoo80,zizooe,zksdk,zksvk,zpsemf925,zq3824,zsw6670
title,,,,,,,,,,,,,,,,,,,,,
BMW 1시리즈 (F20),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BMW 3시리즈 (F30),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
BMW 5시리즈 (F10),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
BMW 5시리즈 (G30),0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0
BMW M2 (F87),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 차량과 차량들 간 코사인 유사도 산출

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

car_cos_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
car_cos_sim

array([[1.        , 0.13305139, 0.04393368, ..., 0.14405508, 0.        ,
        0.28077997],
       [0.13305139, 1.        , 0.06131864, ..., 0.10052949, 0.        ,
        0.19594357],
       [0.04393368, 0.06131864, 1.        , ..., 0.06638985, 0.        ,
        0.16637333],
       ...,
       [0.14405508, 0.10052949, 0.06638985, ..., 1.        , 0.        ,
        0.24245555],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.01484798],
       [0.28077997, 0.19594357, 0.16637333, ..., 0.24245555, 0.01484798,
        1.        ]])

In [19]:
# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
# array -> dataframe
car_cos_sim_df = pd.DataFrame(data=car_cos_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(car_cos_sim_df.shape)
car_cos_sim_df.head(5)

(176, 176)


title,BMW 1시리즈 (F20),BMW 3시리즈 (F30),BMW 5시리즈 (F10),BMW 5시리즈 (G30),BMW M2 (F87),BMW Z4 (E89),기아 K3,기아 K5,기아 K5 2세대,기아 K5 3세대,...,현대 올 뉴 투싼,현대 제네시스,현대 제네시스 DH,현대 제네시스 쿠페,현대 코나,현대 코나 하이브리드,현대 투스카니,현대 투싼 ix,현대 팰리세이드,혼다 New CR-V
title,,,,,,,,,,,,,,,,,,,,,
BMW 1시리즈 (F20),1.000000,0.133051,0.043934,0.160594,0.0,0.079064,0.231340,0.130330,0.174178,0.0,...,0.080694,0.117741,0.129002,0.000000,0.095928,0.0,0.075200,0.144055,0.0,0.280780
BMW 3시리즈 (F30),0.133051,1.000000,0.061319,0.116147,0.0,0.073567,0.161441,0.090951,0.121551,0.0,...,0.056313,0.082166,0.090024,0.087890,0.066944,0.0,0.052479,0.100529,0.0,0.195944
BMW 5시리즈 (F10),0.043934,0.061319,1.000000,0.060555,0.0,0.036438,0.106616,0.060064,0.042812,0.0,...,0.000000,0.054263,0.092481,0.000000,0.044210,0.0,0.000000,0.066390,0.0,0.166373
BMW 5시리즈 (G30),0.160594,0.116147,0.060555,1.000000,0.0,0.048434,0.106288,0.059879,0.117371,0.0,...,0.074149,0.054095,0.059269,0.057864,0.088147,0.0,0.083908,0.066185,0.0,0.129003
BMW M2 (F87),0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [28]:
# 포르쉐 911 와(과) 유사한 차량 6대 확인해보기 / 내림차순
car_cos_sim_df["포르쉐 911"].sort_values(ascending=False)[:6]

title
포르쉐 911            1.000000
포르쉐 마칸             0.343256
벤츠 GLC-클래스 X253    0.286714
볼보 V40 크로스컨트리      0.279618
르노삼성 QM5           0.279618
혼다 New CR-V        0.254284
Name: 포르쉐 911, dtype: float64

### CF-KNN : 아이템 기반 인접 이웃 협업 필터링으로 개인화된 차량 추천 (가중 예측평점)

In [29]:
# 평점 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 평점을 계산하는 함수 정의
def predict_rating(ratings_arr, car_cos_sim_arr):
    ratings_pred = ratings_arr.dot(car_cos_sim_arr)/ np.array([np.abs(car_cos_sim_arr).sum(axis=1)])
    return ratings_pred

In [30]:
ratings_matrix.values   # 사용자-아이템 평점 행렬

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
car_cos_sim_df.values   # 코사인 유사도 데이터프레임

array([[1.        , 0.13305139, 0.04393368, ..., 0.14405508, 0.        ,
        0.28077997],
       [0.13305139, 1.        , 0.06131864, ..., 0.10052949, 0.        ,
        0.19594357],
       [0.04393368, 0.06131864, 1.        , ..., 0.06638985, 0.        ,
        0.16637333],
       ...,
       [0.14405508, 0.10052949, 0.06638985, ..., 1.        , 0.        ,
        0.24245555],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.01484798],
       [0.28077997, 0.19594357, 0.16637333, ..., 0.24245555, 0.01484798,
        1.        ]])

In [32]:
ratings_pred = predict_rating(ratings_matrix.values , car_cos_sim_df.values) # 사용자-아이템 평점 행렬 * 코사인유사도값 
ratings_pred

array([[0.04215487, 0.05886277, 0.02421391, ..., 0.02800368, 0.        ,
        0.04306893],
       [0.02498555, 0.02325898, 0.        , ..., 0.        , 0.        ,
        0.02188056],
       [0.02741489, 0.02644845, 0.02576816, ..., 0.00993373, 0.        ,
        0.01527782],
       ...,
       [0.03613206, 0.02501476, 0.01567265, ..., 0.01812561, 0.        ,
        0.02751467],
       [0.11177925, 0.13751922, 0.51121826, ..., 0.05272336, 0.        ,
        0.08546571],
       [0.02741489, 0.02644845, 0.02576816, ..., 0.00993373, 0.        ,
        0.01527782]])

In [34]:
# 데이터프레임으로 변환
ratings_pred_df = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
print(ratings_pred_df.shape)
ratings_pred_matrix.head(5)

(1034, 176)


title,BMW 1시리즈 (F20),BMW 3시리즈 (F30),BMW 5시리즈 (F10),BMW 5시리즈 (G30),BMW M2 (F87),BMW Z4 (E89),기아 K3,기아 K5,기아 K5 2세대,기아 K5 3세대,...,현대 올 뉴 투싼,현대 제네시스,현대 제네시스 DH,현대 제네시스 쿠페,현대 코나,현대 코나 하이브리드,현대 투스카니,현대 투싼 ix,현대 팰리세이드,혼다 New CR-V
id,,,,,,,,,,,,,,,,,,,,,
Baestyle,0.042155,0.058863,0.024214,0.046945,0.0,0.045993,0.032969,0.063489,0.034072,0.000000,...,0.025843,0.024985,0.023912,0.000000,0.016882,0.0,0.038347,0.028004,0.0,0.043069
Bestsoul4,0.024986,0.023259,0.000000,0.017574,0.0,0.000000,0.000000,0.000000,0.018849,0.000000,...,0.030635,0.000000,0.000000,0.000000,0.000000,0.0,0.024477,0.000000,0.0,0.021881
Bosshuga,0.027415,0.026448,0.025768,0.261334,0.0,0.014502,0.011695,0.010510,0.017727,0.000000,...,0.018335,0.008863,0.008482,0.063746,0.011977,0.0,0.017788,0.009934,0.0,0.015278
Button0308,0.030056,0.027979,0.017264,0.031709,0.0,0.021861,0.023506,0.021124,0.051825,0.027847,...,0.036851,0.017814,0.017049,0.000000,0.024074,0.0,0.056785,0.019966,0.0,0.030707
Chdnet,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000


### 예측 평점 정확도를 판단하기 위해 오차 함수인 RMSE를 이용

In [35]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


아이템 기반 모든 인접 이웃 MSE:  10.10572699658128


### 차량 추천

In [43]:
# 임의의 사람(encar.csv 속 id)에게 차량를 추천해보자   ( 임의의 사람 : kjhgytr )
# 추천에 앞서 임의의 사람이 높은 평점을 준 영화를 확인해보면
user_rating_id = ratings_matrix.loc['kjhgytr', :]     # kjhgytr <ㅡ 이 사람이 남긴 평점 확인
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]    # 0보다 크다는 건 평점을 줬다는 것(고의로 0점 준 것 제외)

#우연히 kjhgytr <ㅡ 이 사람은 리뷰를 많이 작성했으므로 10개의 결과가 나오지만, 1~2개의 리뷰를 남긴 사람은 결과가 적다

title
BMW 1시리즈 (F20)    0.0
BMW 3시리즈 (F30)    0.0
BMW 5시리즈 (F10)    0.0
BMW 5시리즈 (G30)    0.0
BMW M2 (F87)      0.0
                 ... 
현대 코나 하이브리드       3.0
현대 투스카니           4.0
현대 투싼 ix          0.0
현대 팰리세이드          0.0
혼다 New CR-V       0.0
Name: kjhgytr, Length: 176, dtype: float64

In [ ]:
# 리뷰를 남기지 않은 차량 중에 추천
# user_rating이 0보다 크면 기존에 관람한 영화라는 점을 이용해서 계산

In [54]:
def get_unview_car(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 차량정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 차량명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 리뷰한 차량임. 대상 index를 추출하여 list 객체로 만듬
    already_viewed = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 차량명을 list 객체로 만듬. 
    car_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_viewed에 해당하는 차량은 car_list에서 제외함. 
    unview_list = [ car for car in car_list if car not in already_viewed]
    
    return unview_list


In [55]:
# pred_df : 앞서 계산된 영화 별 예측 평점
# unview_list : 사용자가 보지 않은 차량들
# top_n : 상위 n개를 가져온다.

def recomm_car_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unview_list로 들어온 차량명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_car = pred_df.loc[userId, unview_list].sort_values(ascending=False)[:top_n]
    return recomm_car

In [56]:
# 사용자가 보지 않은 차량명 추출   
unview_list = get_unview_car(ratings_matrix, 'kjhgytr' ) 

# 아이템 기반의 인접 이웃 협업 필터링으로 차량 추천 
recomm_car = recomm_car_by_userid(ratings_pred_df, 'kjhgytr' , unview_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_car_df = pd.DataFrame(data=recomm_car.values,index=recomm_car.index, columns=['pred_score'])
recomm_car_df

,pred_score
title,
기아 스포티지 더 볼드,0.898578
르노삼성 더 뉴 SM6,0.822791
쉐보레(GM대우) 콜로라도,0.801462
쉐보레(GM대우) 트래버스,0.764515
현대 더 뉴 아반떼 AD,0.759467
쉐보레(GM대우) 올 뉴 크루즈,0.757596
현대 그랜저 뉴 럭셔리,0.752694
제네시스 G80,0.731140
현대 아반떼 (CN7),0.728666


In [57]:
# 'kjhgytr' <- 이 사람이 리뷰를 남긴 차량 중 top10
user_rating_id = ratings_matrix.loc['kjhgytr', :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

title
현대 제네시스 DH           5.0
쉐보레(GM대우) 트레일블레이저    4.5
현대 투스카니              4.0
현대 베라크루즈             4.0
현대 아슬란               4.0
현대 쏘나타 더 브릴리언트       3.0
현대 싼타페 DM            3.0
현대 싼타페 TM            3.0
현대 싼타페 더 프라임         3.0
현대 쏘나타 (DN8)         3.0
Name: kjhgytr, dtype: float64